In [ ]:
# want to see the images inline
# %matplotlib inline

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy.stats import gamma
from scipy.optimize import curve_fit
from scipy import special

from configparser import ConfigParser

import utils as utils

from tqdm import tqdm_notebook as tqdm

In [ ]:
# read config file 
config = ConfigParser()
config.read('config.ini')

In [ ]:
FOLDER = config.get('all', 'FOLDER')
DB_FILENAME = config.get('all', 'DB_FILENAME')
DB_NEW_FILENAME = config.get('all', 'DB_NEW_FILENAME')
EXPERIMENTS = ['N2', 'SEA-12', 'MK4', 'CB428', 'RNAi']
CHANNELS = ['C0-', 'C1-', 'C2-', 'C3-', 'C4-']
SMFISH_COLUMNS = ['#c0_smfish', '#c1_smfish', '#c2_smfish', '#c3_smfish', '#c4_smfish']

In [ ]:
# read the db and parse images that we want to process
df_path = os.path.join(FOLDER, DB_FILENAME)
df = pd.read_csv(df_path, 
                 sep=',',
                 na_values=[''],
                )
df.head()

In [ ]:
# TODO: maybe it is necessary to fill in other values here, too
# fix missing entries in the colmns that we are planning to use 
df['cropped_image_file'].fillna('', inplace=True)
df['c0_type'].fillna('', inplace=True)
df['c1_type'].fillna('', inplace=True)
df['c2_type'].fillna('', inplace=True)

In [ ]:
for c in SMFISH_COLUMNS:
    df[c] = np.nan

In [ ]:
pbar = tqdm(total=len(df))
for i, r in df.iterrows():
    pbar.update(1)
    filename = r['cropped_image_file']
    if (filename == ''):
        continue
        
    e = filename.split('_')[0]
    folder = os.path.join(FOLDER, e, 'csv-2')
    
    for c in CHANNELS:
        filepath = os.path.join(folder, c + filename[:-4] + '.csv')
        if(not os.path.exists(filepath)):
            continue

        num_lines = sum(1 for line in open(filepath))
        smfish_column = '#' + c.lower()[:2] + '_smfish'
        # print(f'{smfish_column}: {num_lines}')

        df.at[i, smfish_column] = num_lines
pbar.close()    

In [ ]:
df.head()

In [ ]:
df.to_csv(os.path.join(FOLDER, DB_NEW_FILENAME), 
          index=False,
         )